## Tekstide indekseerimine

Järgnevas kasutame tekstide indekseerimiseks spetysiifilist veebiteenust, mis väljastab peale tekstides sisalduvate sõnade algvormide palju muud morfoloogilist informatsiooni, mida läheb meil edaspidi päringulaiendaja loomiseks vaja sealhulgas ka sünavormide asukohti tekstis.


Dokumentide indekseerimiseks võib analüüsida kas originaaltekste või neile vastavaid puhastatud tekste. 
Puhastatud tekstide analüüsimisel väljundindeksis olevad sõnavormide asukohad ei pruugi vastata sõnade tegelikule asukohale originaaltekstis, sest teksti täiendamine lisab teksti sümboleid. 

Kuna meie lõppeesmärk indeksi loomisel on vaid otsitavate sõnavormide leidmine mitte nende täpse asukoha talletamine, siis me ignoreerime antud probleemi ja kasutame indekseerimiseks puhastatud tekste.
Vajadusel saaks sõnavormide asukohad ümber arvutada, jättes meelde originaalteksti ja puhastatud teksti sõnede omavahelise vastavuse. 


In [18]:
import json
import math
import requests

from pandas import DataFrame 
from pandas import read_csv
from tqdm.auto import tqdm
from typing import List

## I. Indekseerimisteenuse dokumenthaaval kasutamine

Veebiteenuse kasutamine üks dokument korraga on sobilik juhul, kui dokumendid on pikad või on neid piisavalt palju, et ühe liitpäringu tegemine on liiga koormav. Näiteks siis, kui indekseeritakse seaduse tekste. 
Kuna järgnevad sammud eeldavad, et väljundindeksis olev `DOCID` väli vastab dokumendi globaalsele indeksile, siis päringu tegemisel tuleb määrata nii tekst kui ka sellele vastav `global_id` väli Riigi Teataja infosüsteemis. 



In [2]:
def index_document(doc_id: str, text: str, disambiguation: bool = False):
    """
    Uses web service to index document for further processing.

    Returns a JSON object that can be further modified or serialised to text.
    All indices are inside the field 'tabelid' which contains seven subfields.

    Two out of these correspond to actual word locations:
    - Subfield 'indeks_lemmad' contains information about lemmas in an unspecified order. 
    - Subfield 'indeks_vormid' contains information about wordforms in an unspecified order. 

    The remaining four contain aggregated information about the document:  
    - Subfield 'liitsõnad' contains what subwords compound words in the document contain. 
    - Subfield 'kirjavead' contains potential spelling mistakes for each wordform.
    - Subfield 'lemma_kõik_vormid' contains all potential wordform for each lemma.
    - Subfield 'lemma_korpuse_vormid' contains all wordform for each lemma that exists in the document.
    """
        
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/create_jsontables/document"
    HEADERS = {"Content-Type": "application/json"}
    POST_DATA_TEMPLATE = {"sources": {str(doc_id): {"content": text}}}
    
    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    return response.json()

In [3]:
text = \
"""
Kõikumatus usus ja vankumatus tahtes kindlustada ja arendada riiki,
mis on loodud Eesti rahva riikliku enesemääramise kustumatul õigusel ja välja kuulutatud 1918. aasta 24. veebruaril,
mis on rajatud vabadusele, õiglusele ja õigusele,
mis on kaitseks sisemisele ja välisele rahule ning pandiks praegustele ja tulevastele põlvedele nende ühiskondlikus edus ja üldises kasus,
mis peab tagama eesti rahvuse, keele ja kultuuri säilimise läbi aegade –
võttis Eesti rahvas 1938. aastal jõustunud põhiseaduse § 1 alusel 1992. aasta 28. juuni rahvahääletusel vastu järgmise põhiseaduse.
"""
result = index_document(115052015002, text)

### Lemma indeksi kirjeldus näidetena

Kuna ühel sõnal võib olla mitu võimalikku algvormi, siis ühele lokatsioonile võib vastata mitu lemmat.

In [4]:
lemma_index = DataFrame(result['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'is_sublemma'])
display(lemma_index.sort_values('start').head(5))
display(lemma_index.sort_values('start').head(5))

,lemma,doc_id,start,end,is_sublemma
91,kõikumatu,115052015002,1,11,False
89,kõikumatus,115052015002,1,11,False
22,usk,115052015002,12,16,False
55,vankumatus,115052015002,20,30,False
19,vankumatu,115052015002,20,30,False


,lemma,doc_id,start,end,is_sublemma
91,kõikumatu,115052015002,1,11,False
89,kõikumatus,115052015002,1,11,False
22,usk,115052015002,12,16,False
55,vankumatus,115052015002,20,30,False
19,vankumatu,115052015002,20,30,False


In [5]:
print(f"Lemma:           {lemma_index.loc[0, 'lemma']}")
print(f"Lokatsioon:      [{lemma_index.loc[0, 'start']}, {lemma_index.loc[0, 'end']})")
print(f"Vastav sõnavorm: {text[lemma_index.loc[0, 'start']:lemma_index.loc[0, 'end']]}")

Lemma:           arendama
Lokatsioon:      [53, 61)
Vastav sõnavorm: arendada


In [6]:
lemma_index[lemma_index['is_sublemma']].style.set_caption("Liitsõna alamosad")

,lemma,doc_id,start,end,is_sublemma
24,määramine,115052015002,104,118,True
25,seadus,115052015002,491,502,True
36,põhi,115052015002,567,578,True
47,põhi,115052015002,491,502,True
59,hääletus,115052015002,536,551,True
61,rahvas,115052015002,536,551,True
63,seadus,115052015002,567,578,True
93,ise,115052015002,104,118,True


In [7]:
print(f"Lemma:           {lemma_index.loc[36, 'lemma']}")
print(f"Lokatsioon:      [{lemma_index.loc[36, 'start']}, {lemma_index.loc[36, 'end']})")
print(f"Vastav sõnavorm: {text[lemma_index.loc[36, 'start']: lemma_index.loc[36, 'end']]}")

Lemma:           põhi
Lokatsioon:      [567, 578)
Vastav sõnavorm: põhiseaduse


### Sõnavormide indeksi kirjeldus näidetena

Sõnavormide pulul võimalikku mitmesust ei ole. Ühele lokatsioonile vastab ainult üks sõnavorm.

In [8]:
wordform_index = DataFrame(result['tabelid']['indeks_vormid'], columns=['wordform', 'doc_id', 'start', 'end', 'is_sublemma'])
display(wordform_index.head(5))
display(wordform_index.sort_values('start').head(5))

,wordform,doc_id,start,end,is_sublemma
0,võttis,115052015002,448,454,False
1,põhiseaduse,115052015002,491,502,False
2,aastal,115052015002,474,480,False
3,1,115052015002,505,506,False
4,keele,115052015002,406,411,False


,wordform,doc_id,start,end,is_sublemma
56,kõikumatus,115052015002,1,11,False
75,usus,115052015002,12,16,False
24,vankumatus,115052015002,20,30,False
9,tahtes,115052015002,31,37,False
16,kindlustada,115052015002,38,49,False


In [9]:
print(f"Lemma:           {wordform_index.loc[1, 'wordform']}")
print(f"Lokatsioon:      [{wordform_index.loc[1, 'start']}, {wordform_index.loc[1, 'end']})")
print(f"Vastav sõnavorm: {text[wordform_index.loc[1, 'start']:wordform_index.loc[1, 'end']]}")


Lemma:           põhiseaduse
Lokatsioon:      [491, 502)
Vastav sõnavorm: põhiseaduse


In [10]:
display(wordform_index[wordform_index['start'] == 536].sort_values('wordform').style.set_caption("Rahvahääletuse alamsõnavormid"))
display(lemma_index[lemma_index['start'] == 536].sort_values('lemma').style.set_caption("Rahvahääletuse alamsõnavormide lemmad"))

,wordform,doc_id,start,end,is_sublemma
36,hääletusel,115052015002,536,551,True
32,rahva,115052015002,536,551,True
70,rahvahääletusel,115052015002,536,551,False


,lemma,doc_id,start,end,is_sublemma
59,hääletus,115052015002,536,551,True
60,rahvahääletus,115052015002,536,551,False
61,rahvas,115052015002,536,551,True


### Liitsõnade tabeli kirjeldus näidetena 

Kuna ühel sõnal võib olla mitu võimalikku algvormi, siis liitsõna eri osadele võib vastata mitu lemmat.

In [11]:
compound_words = DataFrame(result['tabelid']['liitsõnad'], columns=['sublemma', 'lemma'])
display(compound_words.sort_values('lemma'))

,sublemma,lemma
3,ise,enesemääramine
5,määramine,enesemääramine
1,põhi,põhiseadus
2,seaduma,põhiseadus
7,seadus,põhiseadus
0,hääletus,rahvahääletus
4,rahvas,rahvahääletus
6,hääletu,rahvahääletus


### Võimalike kirjavigade tabeli kirjeldus näidetena

Teades tekstides olevaid sõnavorme ning tüüpilisemaid viise kuidas inimesed kirjavigu teevad saab leida, millised vigaste otsisõnadega päringud peaksid andma tulemuse. Kuna pole võimalik ette näha, mis vormis otsisõna kirjutatud on, siis on tabelis kõikvõimalikud sõnavormid, mis on saadud tekstides esinevate sõnade algvormide käänamisest ja pööramisest. Oluline on tähele panna, et üks vigane vorm võib vastata mitmele korrektsele sõnavormile ning isegi mitmele korrektsele algvormile.

In [12]:
misspellings = DataFrame(result['tabelid']['kirjavead'], columns=['misspelling', 'wordform'])
display(misspellings.sort_values('misspelling').head(5))

,misspelling,wordform
52023,EEesti,Eesti
8552,EEestiga,Eestiga
64834,EEestiks,Eestiks
34762,EEestil,Eestil
63410,EEestile,Eestile


In [13]:
display(misspellings
        .groupby('misspelling')
        .aggregate(count=('misspelling', len), misspelling=('wordform',lambda x: ', '.join(set(x))))
        .pipe(lambda df: df[df['count'] > 1])
        .sample(5)
        .style.set_caption("Mitmele sõnavormile vastavad kirjavead"))

,count,misspelling
misspelling,,
arendat,4,"arendate, arendata, arendati, arendad"
jõustunutle,2,"jõustunutele, jõustunutel"
kaitsa,2,"kaitsma, kaitsta"
taaksite,2,"tagaksite, tahaksite"
keelsse,2,"keelisse, keelesse"


### Lemmade kõikide vormide tabeli kirjeldus näidetena

Selles tabelis on kõikvõimalikud tekstides esinevate sõnade algvormide käänamisel ja pööramisel saadud vormid. Tabelis olev kaal näitab sõnavormi sagedust tekstides.

In [14]:
all_wordforms = DataFrame(result['tabelid']['lemma_kõik_vormid'], columns=['wordform', 'weight', 'lemma'])
display(all_wordforms.sort_values('weight', ascending=False).head(5))

,wordform,weight,lemma
768,on,3,olema
2853,eesti,3,eesti
1093,eesti,3,Eesti
1036,Eesti,2,Eesti
2144,rahva,2,rahvas


### Lemmade kõikide tekstis esinevate vormide tabeli kirjeldus näidetena

Selles tabelis on kõikvõimalikud tekstides esinevate sõnavormide sagedus.


In [15]:
existing_wordforms = DataFrame(result['tabelid']['lemma_korpuse_vormid'], columns=['wordform', 'weight', 'lemma'])
display(existing_wordforms.sort_values('weight', ascending=False).head(5))

,wordform,weight,lemma
61,eesti,3,eesti
65,Eesti,3,eesti
58,olema,3,on
5,aasta,2,aasta
6,seadus,2,seaduse


## II. Indekseerimisteenuse kasutamine mitme dokumendi jaoks

Sama veebiteenust saab kasutada ka mitme dokumandi korraga indekseerimiseks. Seda on mõistlik teha vähendamaks veebipäringute arvu ja väljundite summaarse andmemahu vähendamiseks. Ainsaks probleemiks on sisendi suurus. Vaikimisi on veebiteenus seadistatud nii, et kogusisendi suurus ei tohiks olla üle 10 megabaidi. 

**Sisendi suuruse muutmine:** TODO

In [19]:
def index_documents(doc_ids: List[str], texts: List[str], disambiguation: bool = False, size_limit:int = 10 * 10**6):
    """
    Uses web service to index documents for further processing.
    Raises value error if the input creates too long input for the webservice.
    Size limit is given in bytes.

    Returns a JSON object that can be further modified or serialised to text.
    All indices are inside the field 'tabelid' which contains seven subfields.

    Two out of these correspond to actual word locations:
    - Subfield 'indeks_lemmad' contains information about lemmas in an unspecified order. 
    - Subfield 'indeks_vormid' contains information about wordforms in an unspecified order. 

    The remaining four contain aggregated information about the document:  
    - Subfield 'liitsõnad' contains what subwords compound words in the document contain. 
    - Subfield 'kirjavead' contains potential spelling mistakes for each wordform.
    - Subfield 'lemma_kõik_vormid' contains all potential wordform for each lemma.
    - Subfield 'lemma_korpuse_vormid' contains all wordform for each lemma that exists in the document.
    """
        
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/create_jsontables/document"
    HEADERS = {"Content-Type": "application/json"}
    POST_DATA_TEMPLATE = {'sources': {id: {'content': text} for id, text in zip(doc_ids, texts)}} 

    # Abort if the input is too long
    byte_size =len(json.dumps(POST_DATA_TEMPLATE, ensure_ascii=False).encode("utf-8"))
    if byte_size > size_limit:
        raise ValueError('Input is too long')

    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"

    return response.json()

In [20]:
doc_ids=['25055', '964508', '202122020001', '123042022001']
texts = ["Rahvusvahelise patendiklassifikatsiooni Strasbourg'i kokkuleppega ühinemise seadus", 
         "Tuumaohutuse konventsiooniga ühinemise seadus",
         "Eesti Vabariigi ja Euroopa Tuumauuringute Organisatsiooni(CERN) vahelise CERN-iga "
         "ühinemise eelses staadiumis assotsieerunud liikme staatuse andmist käsitleva kokkuleppe ratifitseerimise seadus",
         "Seadus «Riigi 2002. aasta lisaeelarve»"] 

result = index_documents(doc_ids, texts)

In [21]:
wordform_index = DataFrame(result['tabelid']['indeks_vormid'], columns=['wordform', 'doc_id', 'start', 'end', 'is_sublemma'])
display(wordform_index.sort_values('wordform')[:10])

,wordform,doc_id,start,end,is_sublemma
7,2002.,123042022001,14,19,False
42,CERN,202122020001,58,62,False
17,CERNiga,202122020001,73,81,False
12,Eesti,202122020001,0,5,False
46,Euroopa,202122020001,19,26,False
14,Strasbourg’,25055,40,52,False
20,aasta,123042022001,20,25,False
38,andmist,202122020001,141,148,False
48,arve,123042022001,26,37,True
49,assotsieerunud,202122020001,110,124,False


In [22]:
lemma_index = DataFrame(result['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'is_sublemma'])
display(lemma_index.sort_values('lemma')[:10])

,lemma,doc_id,start,end,is_sublemma
8,2002.,123042022001,14,19,False
44,CERN,202122020001,73,81,False
56,CERN,202122020001,58,62,False
17,Eesti,202122020001,0,5,False
59,Euroopa,202122020001,19,26,False
40,Strasbourg,25055,40,52,False
30,aasta,123042022001,20,25,False
39,andmine,202122020001,141,148,False
28,arv,123042022001,26,37,True
55,arve,123042022001,26,37,True


**Näide pikkusepiirangu rakendumisest**

In [23]:
try:
    index_documents(doc_ids, texts, size_limit=10)
except ValueError as e:
    print(e)

Input is too long


## III. Riigiteataja pealkirjade indekseerimine

Lihtsuse ja selguse mõttes indekseerime dokumente järjestikku kombineerides dokumentide pealkirjad 1000 elemendilisteks gruppideks ning salvestame tulemused kataloogi `results/document_indeks`. Praktikas oleks suuremate mahtude korral mõistlik teha indekseerimist paraleelselt nii, et igale skriptile vastaks oma veebiteenuse lõim. Näiteid selle kohta leiab [smart-search repost](???).

In [24]:
sources = {}
sources['state_laws'] = read_csv('../results/cleaned_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/cleaned_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/cleaned_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/cleaned_texts/government_orders.csv', header=0)

In [51]:
BATCH_SIZE = 1000
OUTPUT_DIR = '../results/document_index/'
for name, source in sources.items():
    print(f'Sisendfail: {name}.csv ({len(source)} rida)')
    
    batch_count = math.ceil(len(source)/BATCH_SIZE)
    for k in tqdm(range(batch_count)):
        batch = source.loc[k*BATCH_SIZE:(k+1)*BATCH_SIZE, ['global_id', 'document_title']]
        #result = index_documents(batch['global_id'], batch['document_title'])
        result = {'kala': 5} 
        with open(f'{OUTPUT_DIR}{name}_{k:03d}.json', 'w') as output:
            json.dump(result, output)    

Sisendfail: state_laws.csv (4730 rida)


  0%|          | 0/5 [00:00<?, ?it/s]

Sisendfail: government_regulations.csv (12609 rida)


  0%|          | 0/13 [00:00<?, ?it/s]

Sisendfail: local_government_acts.csv (36525 rida)


  0%|          | 0/37 [00:00<?, ?it/s]

Sisendfail: government_orders.csv (4973 rida)


  0%|          | 0/5 [00:00<?, ?it/s]

### Uute pealkirjade indekseerimine

Uute dokumentide lisandumisel tekib vajadus neid indekseerida. Seda saab teha analoogselt pealkirjade indekseerimisega ning seejärel lisada uued andmed päringulaiendajasse. Seda saab põhimõtteliselt teha kahel moel. Esiteks võib teha kogu analüüsi uuest. Teiseks võib teha analüüsi vaid uute dokumentide peal. Indekseerimisväljund on selline, et seda saab kasutada otse olemasoleva päringulaiendaja sisendi uuendamiseks. 

